In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Sales Data Analysis").getOrCreate()

# File location and type
file_location = "/Users/inbravo/Documents/GitHub/python-feature-set/dataset/super-market/sales-less-record.csv"

# Read the CSV data from the file
df = spark.read.format("csv") \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", ",") \
  .load(file_location)

# Show the first 5 rows of the DataFrame
df.show(5)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/10 18:55:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice_ID|Branch|     City|Customer_Type|Gender|        Product_Line|Unit_Price|Quantity| Tax_5%|   Total|     Date|               Time|    Payment|  cogs|Gross_Margin_Percentage|Gross_Income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715| 1/5/2019|2025-05-10 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22| 3/8/2019|2025-05-10 10:29:00|       Cash|  76.4|            4.761904762|      

In [2]:
from pyspark.sql.functions import round
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import IntegerType
   
# Typecast the 'Gross income' values to Integer type    
df = df.withColumn("Gross_Income", df["Gross_Income"].cast(IntegerType()))

# Round gross income to 2 decimal places
df = df.withColumn("Gross_Income", round(df["Gross_Income"], 2))

# Group by the "Product line"
df = df.groupBy("Product_Line")

# Sum of the gross income
gross_income_sum = _sum("Gross_Income")

# Sum the gross income
df_round = df.agg(gross_income_sum).alias("Gross_Income")

# Show the first 5 rows of the DataFrame
df_round.show(5)

+--------------------+-----------------+
|        Product_Line|sum(Gross_Income)|
+--------------------+-----------------+
|  Home and lifestyle|               67|
| Fashion accessories|               30|
|   Health and beauty|               28|
|Electronic access...|                3|
|  Food and beverages|                1|
+--------------------+-----------------+

